## Uurime K-Means klasterdamist R-i ja Tidy andmete põhimõtete abil.

### [**Eelloengu viktoriin**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Selles õppetükis õpid, kuidas luua klastreid, kasutades Tidymodels paketti ja teisi R-i ökosüsteemi pakette (nimetame neid sõpradeks 🧑‍🤝‍🧑) ning varem imporditud Nigeeria muusika andmestikku. Käsitleme K-Meansi klasterdamise põhialuseid. Pea meeles, et nagu sa eelmises õppetükis õppisid, on klastritega töötamiseks palju erinevaid viise ja meetod sõltub sinu andmetest. Proovime K-Meansi, kuna see on kõige levinum klasterdamise tehnika. Alustame!

Mõisted, mida sa õpid:

-   Silueti skoorimine

-   Küünarnuki meetod

-   Inerts

-   Variantsus

### **Sissejuhatus**

[K-Meansi klasterdamine](https://wikipedia.org/wiki/K-means_clustering) on meetod, mis pärineb signaalitöötluse valdkonnast. Seda kasutatakse andmete rühmade jagamiseks ja jaotamiseks `k klastrisse`, lähtudes nende omaduste sarnasustest.

Klastreid saab visualiseerida [Voronoi diagrammidena](https://wikipedia.org/wiki/Voronoi_diagram), mis sisaldavad punkti (või 'seemet') ja selle vastavat piirkonda.

<p >
   <img src="../../../../../../translated_images/et/voronoi.1dc1613fb0439b95.png"
   width="500"/>
   <figcaption>Infograafik: Jen Looper</figcaption>


K-Meansi klasterdamisel on järgmised sammud:

1.  Andmeteadlane määrab esmalt soovitud klastrite arvu, mida luua.

2.  Seejärel valib algoritm juhuslikult K vaatlust andmestikust, et need saaksid olla klastrite esialgseteks keskusteks (st tsentroidideks).

3.  Seejärel määratakse iga ülejäänud vaatlus lähimale tsentroidile.

4.  Seejärel arvutatakse iga klastri uus keskmine ja tsentroid nihutatakse keskmise asukohta.

5.  Kui keskused on ümber arvutatud, kontrollitakse iga vaatlust uuesti, et näha, kas see võiks olla lähemal mõnele teisele klastrile. Kõik objektid määratakse uuesti, kasutades uuendatud klastri keskmisi. Klastri määramise ja tsentroidi uuendamise samme korratakse iteratiivselt, kuni klastri määramised enam ei muutu (st kui saavutatakse konvergents). Tavaliselt lõpetab algoritm töö, kui iga uus iteratsioon põhjustab tsentroidide tühise liikumise ja klastrid muutuvad staatiliseks.

<div>

> Pange tähele, et kuna algsete k vaatlustega tsentroidide valik on juhuslik, võivad tulemused iga kord veidi erineda. Seetõttu kasutavad enamik algoritme mitut *juhuslikku algust* ja valivad iteratsiooni, millel on madalaim WCSS. Seetõttu on tungivalt soovitatav alati käivitada K-Means mitme *nstart* väärtusega, et vältida *ebasoovitavat lokaalset optimaalset lahendust.*

</div>

See lühike animatsioon, mis kasutab Allison Horsti [illustratsioone](https://github.com/allisonhorst/stats-illustrations), selgitab klasterdamise protsessi:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Illustratsioon: @allison_horst</figcaption>



Põhiküsimus, mis klasterdamisel tekib, on järgmine: kuidas teada, mitmeks klastriks oma andmed jagada? Üks K-Meansi puudusi on see, et pead määrama `k`, st `tsentroidide` arvu. Õnneks aitab `küünarnuki meetod` hinnata head algväärtust `k` jaoks. Sa proovid seda kohe.

### 

**Eeltingimus**

Jätkame täpselt sealt, kus eelmises [õppetükis](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) pooleli jäime, kus analüüsisime andmestikku, tegime palju visualiseeringuid ja filtreerisime andmestikku huvipakkuvateks vaatlusteks. Kindlasti vaata see üle!

Selle mooduli läbimiseks vajame mõningaid pakette. Sa saad need installida käsuga: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Teise võimalusena kontrollib allolev skript, kas sul on selle mooduli läbimiseks vajalikud paketid olemas, ja installib need, kui mõni neist puudub.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Lähme kohe asja kallale!

## 1. Tants andmetega: Kitsenda 3 kõige populaarsema muusikastiilini

See on kokkuvõte sellest, mida tegime eelmises tunnis. Lõikame ja sorteerime andmeid!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 See läks hästi!

## 2. Rohkem andmete uurimist.

Kui puhtad need andmed on? Kontrollime äärmusväärtusi kasutades kastdiagramme. Keskendume numbrilistele veergudele, kus on vähem äärmusväärtusi (kuigi äärmusväärtusi võiks ka eemaldada). Kastdiagrammid näitavad andmete ulatust ja aitavad valida, milliseid veerge kasutada. Pange tähele, et kastdiagrammid ei näita variatsiooni, mis on oluline hea klasterdatava andmestiku element. Lisalugemiseks vaadake [seda arutelu](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Kastdiagrammid](https://en.wikipedia.org/wiki/Box_plot) kasutatakse numbriliste andmete jaotuse graafiliseks kujutamiseks, seega alustame *valides* kõik numbrilised veerud koos populaarsete muusikastiilidega.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Vaata, kui lihtsaks teeb valikuabiline `where` selle 💁? Uuri teisi sarnaseid funktsioone [siin](https://tidyselect.r-lib.org/).

Kuna me loome kastdiagrammi iga numbrilise tunnuse jaoks ja tahame vältida tsüklite kasutamist, siis vormindame oma andmed *pikemasse* formaati, mis võimaldab meil kasutada `facets`-eid – alagraafikuid, mis kuvavad iga andmealamkogumi eraldi.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Palju pikem! Nüüd on aeg mõneks `ggplot`-iks! Millist `geom`-i me kasutame?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Lihtne-gg!

Nüüd näeme, et need andmed on veidi müra täis: vaadates iga veergu kastdiagrammina, on näha erandväärtusi. Võiksid andmestikku läbi käia ja need erandväärtused eemaldada, kuid see muudaks andmed üsna napiks.

Praegu valime veerud, mida kasutame oma klasterdamise harjutuseks. Valime numbrilised veerud, millel on sarnased vahemikud. Me võiksime kodeerida `artist_top_genre` numbriliseks, kuid jätame selle praegu välja.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. K-means klasterdamise arvutamine R-is

R-is saab k-means klasterdamist arvutada sisseehitatud funktsiooniga `kmeans`, vaata `help("kmeans()")`. Funktsioon `kmeans()` võtab peamise argumendina vastu andmeraami, kus kõik veerud on numbrilised.

Esimene samm k-means klasterdamise kasutamisel on määrata klastrite arv (k), mis lõplikus lahenduses luuakse. Me teame, et andmestikust eraldasime 3 laulužanrit, seega proovime 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Kmeans objekt sisaldab mitmeid informatsiooniosi, mida on hästi selgitatud `help("kmeans()")` abil. Praegu keskendume vaid mõnele neist. Näeme, et andmed on jaotatud 3 klastrisse suurustega 65, 110, 111. Väljund sisaldab ka klastrite keskmeid (keskmisi) 3 grupi jaoks üle 5 muutuja.

Klastri vektor näitab iga vaatluse klastrikuuluvust. Kasutame `augment` funktsiooni, et lisada klastrikuuluvus algsele andmestikule.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Täiuslik, me just jagasime oma andmekogumi kolmeks rühmaks. Aga kui hea on meie klasterdamine 🤷? Vaatame lähemalt `Silhouette'i skoori`.

### **Silhouette'i skoor**

[Silhouette'i analüüsi](https://en.wikipedia.org/wiki/Silhouette_(clustering)) saab kasutada, et uurida klastrite vahelisi eraldusvahemaid. See skoor varieerub vahemikus -1 kuni 1, kusjuures skoor, mis on lähedal 1-le, näitab tihedat ja teistest klastritest hästi eraldatud klastrit. Väärtus, mis on lähedal 0-le, viitab kattuvatele klastritele, kus proovid asuvad naaberklaastrite otsustuspiiri lähedal. [allikas](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Keskmise silhouette'i meetod arvutab vaatluste keskmise silhouette'i erinevate *k* väärtuste jaoks. Kõrge keskmine silhouette'i skoor viitab heale klasterdamisele.

`silhouette` funktsiooni klastrite paketis saab kasutada keskmise silhouette'i laiuse arvutamiseks.

> Silhouette'i saab arvutada mis tahes [kauguse](https://en.wikipedia.org/wiki/Distance "Distance") meetrika abil, näiteks [Eukleidiline kaugus](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") või [Manhattani kaugus](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), mida arutasime [eelmises õppetükis](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Meie skoor on **.549**, seega täpselt keskel. See näitab, et meie andmed ei sobi eriti hästi sellist tüüpi klasterdamiseks. Vaatame, kas saame seda oletust visuaalselt kinnitada. [factoextra pakett](https://rpkgs.datanovia.com/factoextra/index.html) pakub funktsioone (`fviz_cluster()`), et klasterdamist visualiseerida.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Klasterite kattumine näitab, et meie andmed ei sobi eriti hästi sellist tüüpi klasterdamiseks, kuid jätkame siiski.

## 4. Optimaalsete klastrite määramine

K-Meansi klasterdamise puhul tekib sageli oluline küsimus - kui klassi silte pole teada, siis kuidas otsustada, mitmeks klastriks andmed jagada?

Üks viis selle välja selgitamiseks on kasutada andmevalimit, et `luua klasterdamismudeleid` järjest suureneva klastrite arvuga (näiteks 1-10) ja hinnata klasterdamise mõõdikuid, nagu **Silhouette'i skoor.**

Määrame optimaalsete klastrite arvu, arvutades klasterdamise algoritmi erinevate *k* väärtuste jaoks ja hinnates **klastri sisemist ruutude summat** (WCSS). Klastri sisemine ruutude summa (WCSS) mõõdab klasterdamise kompaktsust ja soovime, et see oleks võimalikult väike, kuna madalamad väärtused tähendavad, et andmepunktid on üksteisele lähemal.

Uurime, kuidas erinevad `k` väärtused (1 kuni 10) mõjutavad seda klasterdamist.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Nüüd, kui meil on iga klasterdamisalgoritmi jaoks keskusega *k* kogusumma klastrisisestest ruutudest (tot.withinss), kasutame [küünarnuki meetodit](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), et leida optimaalne klastrite arv. Meetod seisneb WCSS-i (klastrisiseste ruutude summa) graafiku joonistamises klastrite arvu funktsioonina ja [kõvera küünarnuki](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Kõvera küünarnukk") valimises klastrite arvuks, mida kasutada.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Graafik näitab WCSS-i suurt vähenemist (seega suuremat *tihedust*), kui klastrite arv suureneb ühest kaheni, ning veel märgatavat vähenemist kahest kolmeni. Pärast seda on vähenemine vähem väljendunud, mis tekitab graafikus umbes kolme klastri juures `küünarnuki` 💪. See on hea viide sellele, et andmepunktid jagunevad kaheks kuni kolmeks mõistlikult hästi eraldatud klastriks.

Nüüd saame edasi minna ja välja võtta klastrimudeli, kus `k = 3`:

> `pull()`: kasutatakse ühe veeru eraldamiseks
>
> `pluck()`: kasutatakse andmestruktuuride, nagu loendite, indekseerimiseks


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Suurepärane! Vaatame nüüd saadud klastreid. Kas soovid interaktiivsust kasutada `plotly` abil?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Võib-olla oleksime oodanud, et iga klaster (esindatud erinevate värvidega) omaks erinevaid žanre (esindatud erinevate kujunditega).

Vaatame mudeli täpsust.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Selle mudeli täpsus pole halb, kuid pole ka suurepärane. Võib juhtuda, et andmed ei sobi hästi K-Means klasterdamiseks. Need andmed on liiga tasakaalust väljas, liiga vähe korreleeritud ja veergude väärtuste vahel on liiga suur varieeruvus, et klasterdamine toimiks hästi. Tegelikult on tõenäoline, et moodustuvad klastrid on tugevalt mõjutatud või kallutatud kolme žanrikategooria poolt, mille me ülalpool määratlesime.

Sellegipoolest oli see päris õpetlik protsess!

Scikit-learn'i dokumentatsioonis näete, et sellisel mudelil, kus klastrid pole väga selgelt eristatud, on "varieeruvuse" probleem:

<p >
   <img src="../../../../../../translated_images/et/problems.f7fb539ccd80608e.png"
   width="500"/>
   <figcaption>Infograafik Scikit-learn'ist</figcaption>



## **Varieeruvus**

Varieeruvus on defineeritud kui "keskmine ruutude erinevus keskmisest" [allikas](https://www.mathsisfun.com/data/standard-deviation.html). Selle klasterdamisprobleemi kontekstis viitab see andmetele, mille väärtused kipuvad keskmisest liiga palju kõrvale kalduma.

✅ See on suurepärane hetk mõelda kõikidele viisidele, kuidas seda probleemi lahendada. Kohandada andmeid veidi rohkem? Kasutada erinevaid veerge? Kasutada teistsugust algoritmi? Vihje: Proovige [andmeid skaleerida](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), et neid normaliseerida ja testida teisi veerge.

> Proovige seda '[varieeruvuse kalkulaatorit](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', et mõista kontseptsiooni veidi paremini.

------------------------------------------------------------------------

## **🚀Väljakutse**

Veetke selle märkmikuga aega, kohandades parameetreid. Kas suudate mudeli täpsust parandada, puhastades andmeid rohkem (näiteks eemaldades kõrvalekaldeid)? Võite kasutada ka kaalusid, et anda teatud andmeproovidele suurem kaal. Mida veel saate teha, et luua paremaid klastreid?

Vihje: Proovige oma andmeid skaleerida. Märkmikus on kommenteeritud kood, mis lisab standardse skaleerimise, et muuta andmeveerud vahemiku osas üksteisele sarnasemaks. Leiate, et kuigi siluett-skoor langeb, muutub "kink" küünarnuki graafikus sujuvamaks. See on tingitud sellest, et skaleerimata andmed võimaldavad väiksema varieeruvusega andmetel suuremat kaalu kanda. Lugege selle probleemi kohta veidi rohkem [siin](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Loengu järgne viktoriin**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Ülevaade ja iseseisev õppimine**

-   Vaadake K-Means simulaatorit [näiteks seda](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Selle tööriista abil saate visualiseerida näidisandmepunkte ja määrata nende centroidid. Saate muuta andmete juhuslikkust, klastrite arvu ja centroidide arvu. Kas see aitab teil saada aimu, kuidas andmeid saab rühmitada?

-   Vaadake ka [seda K-Means'i juhendit](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) Stanfordist.

Kas soovite proovida oma äsja omandatud klasterdamisoskusi andmekogumitel, mis sobivad hästi K-Means klasterdamiseks? Vaadake:

-   [Treeni ja hinda klasterdamismudeleid](https://rpubs.com/eR_ic/clustering) kasutades Tidymodels'i ja teisi tööriistu

-   [K-Means klasterdamise analüüs](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-Means klasterdamine koos korrastatud andmete põhimõtetega](https://www.tidymodels.org/learn/statistics/k-means/)

## **Ülesanne**

[Proovige erinevaid klasterdamismeetodeid](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## TÄNUD:

[Jen Looper](https://www.twitter.com/jenlooper) originaalse Python'i versiooni loomise eest ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) suurepäraste illustratsioonide loomise eest, mis muudavad R-i sõbralikumaks ja kaasahaaravamaks. Leidke rohkem illustratsioone tema [galeriist](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Head õppimist,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../../../../../translated_images/et/r_learners_sm.e4a71b113ffbedfe.jpeg"
   width="500"/>
   <figcaption>Illustratsioon @allison_horst'ilt</figcaption>



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta arusaamatuste või valesti tõlgenduste eest, mis võivad tekkida selle tõlke kasutamisest.
